In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf

In [2]:
traindir = 'C:/Users/karte/OneDrive/Desktop/VIT/MLProjects/MyMLProjects/Emotion Detector/train'
testdir = 'C:/Users/karte/OneDrive/Desktop/VIT/MLProjects/MyMLProjects/Emotion Detector/test'
train_ds = tf.keras.utils.image_dataset_from_directory( #Using keras library to import testing and training dataset
  traindir,                                              #The images are already preprocessed, so im not doing any preprocessing
  image_size=(48,48)
)
test_ds = tf.keras.utils.image_dataset_from_directory(
  testdir,
  image_size=(48,48)
)
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255.0)

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [4]:
model = tf.keras.Sequential([
    normalization_layer,
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(48,48,3)), 
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7)
]
)
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
mymodel = model.fit(train_ds,validation_data=test_ds,epochs=20)

Epoch 1/20
898/898 [==============================] - 28s 30ms/step - loss: 1.6313 - accuracy: 0.3556 - val_loss: 1.5156 - val_accuracy: 0.4166
Epoch 2/20
898/898 [==============================] - 28s 31ms/step - loss: 1.4104 - accuracy: 0.4590 - val_loss: 1.3567 - val_accuracy: 0.4809
Epoch 3/20
898/898 [==============================] - 30s 33ms/step - loss: 1.3153 - accuracy: 0.4996 - val_loss: 1.3101 - val_accuracy: 0.4987
Epoch 4/20
898/898 [==============================] - 29s 33ms/step - loss: 1.2469 - accuracy: 0.5255 - val_loss: 1.3041 - val_accuracy: 0.4965
Epoch 5/20
898/898 [==============================] - 29s 32ms/step - loss: 1.1838 - accuracy: 0.5493 - val_loss: 1.2686 - val_accuracy: 0.5071
Epoch 6/20
898/898 [==============================] - 27s 30ms/step - loss: 1.1332 - accuracy: 0.5720 - val_loss: 1.2648 - val_accuracy: 0.5103
Epoch 7/20
898/898 [==============================] - 27s 30ms/step - loss: 1.0842 - accuracy: 0.5920 - val_loss: 1.2573 - val_accuracy:

In [2]:
model.save('./Saved Models/mymodel')

NameError: name 'model' is not defined

In [3]:
model = tf.keras.models.load_model('./Saved Models/mymodel')

In [13]:
import cv2 #the library used for image processing
import numpy as np
image = cv2.imread('surprised.jpg',cv2.IMREAD_GRAYSCALE)
image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
resized_image = cv2.resize(image_rgb, (48, 48))  # Resize to match the input shape of your model
preprocessed_image = np.expand_dims(resized_image, axis=0)  # Add an extra dimension for batch size (assuming single image) - idk
# Make a prediction
predictions = model.predict(preprocessed_image)

# Interpret the results
predicted_class = np.argmax(predictions)  # Get the index of the class with the highest probability
class_probabilities = predictions[0]  # Probability distribution across all classes
print(test_ds.class_names[predicted_class])
print(f"Class Probabilities: {class_probabilities}")

1/1 [==============================] - 0s 19ms/step
angry
Class Probabilities: [ 3.4562016 -9.203576   2.8400018 -1.9895085 -5.228275  -2.521459
  1.274344 ]


In [18]:
import time
cap = cv2.VideoCapture(0)
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
while True:

    result, video_frame = cap.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully

    gray_image = cv2.cvtColor(video_frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    face = gray_image
    
    for (x, y, w, h) in faces:
        cv2.rectangle(video_frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
        face = gray_image[y:y+h, x:x+w]

    face = cv2.cvtColor(face, cv2.COLOR_GRAY2RGB)
    face = cv2.resize(face, (48, 48))  # Resize to match the input shape of your model
    face = np.expand_dims(face, axis=0)  # Add an extra dimension for batch size (assuming single image) - idk
    # Make a prediction
    predictions = model.predict(face)
    # Interpret the results
    for (x,y,w,h) in faces:
        cv2.putText(video_frame, test_ds.class_names[predicted_class], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    predicted_class = np.argmax(predictions)  # Get the index of the class with the highest probability
    class_probabilities = predictions[0]  # Probability distribution across all classes
    print(test_ds.class_names[predicted_class])
    print(f"Class Probabilities: {class_probabilities}")

    cv2.imshow(
        "My Face Detection Project", video_frame
    )  # display the processed frame in a window named "My Face Detection Project"
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step
angry
Class Probabilities: [  2.17053    -10.1936865    1.9832586   -6.4192147   -0.04938002
  -0.6543417   -3.1372907 ]
1/1 [==============================] - 0s 19ms/step
angry
Class Probabilities: [ 1.4775604  -9.888324    1.2373823  -9.731035    0.839844   -0.23261975
 -0.02976172]
1/1 [==============================] - 0s 19ms/step
neutral
Class Probabilities: [  0.27102202 -11.465931     1.6444529   -7.8092175    3.1376936
  -1.8731148    2.9089096 ]
1/1 [==============================] - 0s 21ms/step
neutral
Class Probabilities: [  0.27102202 -11.465931     1.6444529   -7.8092175    3.1376936
  -1.8731148    2.9089096 ]
1/1 [==============================] - 0s 19ms/step
neutral
Class Probabilities: [  0.62141955 -11.461978     0.8169325   -4.19018      2.4110425
   1.4062808   -1.4705695 ]
1/1 [==============================] - 0s 19ms/step
neutral
Class Probabilities: [  0.62141955 -11.461978     0.8169325   -4.19018      2.4

In [8]:
cap.release()